참고.
https://www.kaggle.com/drvaibhavkumar/alexnet-in-pytorch-cifar10-clas-83-test-accuracy
https://deep-learning-study.tistory.com/376

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import torch.optim as optim

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)



In [ ]:
#CIFAR10 데이터를 이용한 ALEXNET 설계
#1. 하이퍼 파라미터
BATCH_SIZE =  128 #한 배치당 128개 이미지
EPOCHS = 90 
MOMENTUM = 0.9
LR_DECAY =0.0005 #learning rate가 줄어드는 정도.
# 왜필요하지. LR_Scheduler가 있으면?
LR_INIT = 0.01
IMAGE_DIM = 227 #이미지 픽셀 정의
NUM_CLASSES = 1000 #이미지넷의 1000개 이미지 클래스
epoch=0


#모델의 체크포인트 
CHECKPOINT_DIR = '../data/models'


In [ ]:
# 2. 데이터 다운로드. Train Set, Test set 분리
# 미완성.

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(227),##227사이즈로
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),#이미지를 0.5평균, 0.5 sd로 normalization
])

#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True) # 순서가 암기되는것을 막기위해.
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [ ]:
#6. 모델 MLP 설계
#https://deep-learning-study.tistory.com/376
class AlexNet(nn.Module):
    def __init__(self,num_classes = NUM_CLASSES):
        super(AlexNet,self).__init__() # 상속받기
        # input size : (227*227*3)
        #sequential은 한번에 묶어정리
        self.net = nn.Sequential(
            nn.Conv2d(in_channels = 3,
                      out_channels =96,
                      kernel_size = 11,
                      padding =0,
                      stride=4),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5,alpha=0.0001, beta=0.75, k=2), #LRN
            nn.MaxPool2d(kernel_size=3,stride=2),#1번 convlayer 끝. 27x27x96
            nn.Conv2d(in_channels=96,
                      out_channels=256,
                      kernel_size=5,
                      stride=1,
                      padding=2
                      ),# 27x27x256
            nn.ReLU(),
            nn.LocalResponseNorm(size=5,alpha=0.0001, beta=0.75, k=2) ,
            nn.MaxPool2d(kernel_size=3,stride=2),#13x13x256
            #2번 convlayer 끝.
            
            ## convolution 3단
            nn.Conv2d(in_channels=256,
                      out_channels=384,
                      kernel_size=3,
                      stride=1,
                      padding=1),# 13*13*384
            nn.ReLU(),
            nn.Conv2d(in_channels=384,
                      out_channels=384,
                      kernel_size=3,
                      stride=1,
                      padding=1),# 13 * 13 *384
            nn.ReLU(),
            nn.Conv2d(in_channels=384,
                      out_channels=256,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),#6x6x256

        )
        
        #FC Layer설정
        self.classifier = nn.Sequential(
            
            nn.Linear(6*6*256,4096),#인풋 레이어
            nn.ReLU(),
            nn.Dropout(p=0.5,inplace=True),#inplace False면 dropout된 레이어를 직접수정해야한다.
            nn.Linear(4096,4096),#첫번째 히든 레이어
            nn.ReLU(),
            nn.Dropout(p=0.5,inplace=True),
            #두번째 히든레이어 -> 1000개 클래스 아웃
            nn.Linear(4096,num_classes)
            #따로 softMax 함수는 사용하지않음.        
        )
        self.init_bias() # bias 초기화
    
    #논문대로, bias 초기화
    def init_bias(self):
        for layer in self.net:
            if isinstance(layer,nn.Conv2d):
                #Conv layer weight와 bias 초기화
                nn.init.normal_(layer.weight,mean=0,std=0.01)
                nn.init.constant_(layer.bias,0)
        #conv2d 2,4,5만 bias 1로 초기화
        nn.init.constant_(self.net[4].bias,1)
        nn.init.constant_(self.net[10].bias,1)
        nn.init.constant_(self.net[12].bias,1)
        
    
    def forward(self,x):
        x = self.net(x) #먼저 convNet에 넣고
        x = x.view(-1,6*6*256)
        x = x.classifier(x)#fully connected
        return x

In [23]:
#7. 모델생성 ,Optimizer, Objective Function
model = AlexNet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr=LR_INIT,weight_decay=LR_DECAY)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
#30epochs 마다 0.1씩곱해서 lr 감소시키기
print(model)


AlexNet(
  (net): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=Tr

In [ ]:
#8. 학습
def train(model,train_loader,optimizer, log_interval):
    model.train()
    for batch_idx,(image,label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        #데이터들 장비에 할당
        optimizer.zero_grad() # device 에 저장된 gradient 제거
        output = model(image) # model로 output을 계산
        loss = criterion(output, label) #loss 계산
        loss.backward() # loss 값을 이용해 gradient를 계산
        optimizer.step() # Gradient 값을 이용해 파라미터 업데이트.        

In [ ]:
#9. 학습 진행하며, test 데이터로 모델 성능확인
def evaluate(model,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    #no_grad : 그래디언트 값 계산 막기.
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1,keepdim=True)[1] # 가장 확률이 높은 class 1개를 가져온다.그리고 인덱스만
            #print(output.max(1,keepdim=True))
            #print(prediction)
            correct += prediction.eq(label.view_as(prediction)).sum().item()# 아웃풋이 배치 사이즈 32개라서.
            #true.false값을 sum해줌. item
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss,test_accuracy
            

In [ ]:
#10. 학습 및 평가.
seed = torch.initial_seed()
print('Used seed : {}'.foramt(seed))

for Epoch in range(1,EPOCHS+1):
    train(model,train_loader,optimizer,log_interval=200)
    test_loss,test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH:{}], \tTest Loss:{:.4f} \tTest Accuracy: {:.2f} %\n".
          format(Epoch,test_loss,test_accuracy))
    
checkpoint_path = os.path.join(CHECKPOINT_DIR, 'alexnet_states_e{}.pkl'.format(epoch + 1))
state = {
    'epoch': Epoch,
    'optimizer': optimizer.state_dict(),
    'model': model.state_dict(),
    'seed':seed,
}
torch.save(state, checkpoint_path)